Установим требуемые нам библиотеки: 
* `byaldi` - высокоуровневый [фреймворк](https://github.com/AnswerDotAI/byaldi) непосредственно для работы с моделями ColPali
* `pdf2image` - для перевода `.pdf`-файлов в изображения
* `poppler-utils` - для работы `pdf2image`
* `Spire.Doc` - для перевода `.docx`-файлов в формат `.pdf`
* `qwen-vl-utils` - для работы Qwen-VL-моделей

In [1]:
!pip install --upgrade byaldi -q
!sudo apt-get install -y poppler-utils -q
!pip install -q pdf2image flash-attn -q
!pip install Spire.Doc -q
!pip install qwen-vl-utils -q

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libpoppler118 poppler-data
Suggested packages:
  ghostscript fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  libpoppler118 poppler-data poppler-utils
0 upgraded, 3 newly installed, 0 to remove and 72 not upgraded.
Need to get 3427 kB of archives.
After this operation, 17.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 poppler-data all 0.4.11-1 [2171 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler118 amd64 22.02.0-2ubuntu0.5 [1071 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 3427 kB in 0s (23.5 MB/s)
Selecting previously unselected package poppler-data.

Определим папку, где у нас лежат первоначальные данные `input_folder`, а так же папку, куда мы переведём файлы непосредственно для работы с RAG-системой `working_folder`. Так же обозначим папку, где у нас лежит готовая RAG-база `rag_input_dir`, и папку, куда мы положим её для поиска релевантных документов `rag_target_dir`.

In [21]:
from byaldi import RAGMultiModalModel
import torch
from pdf2image import convert_from_path
import os
import shutil
from spire.doc import *
from spire.doc.common import *
import gzip
import json
from PIL import Image, ImageFilter


#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

input_folder = '/kaggle/input/nornikel-2024/'
working_folder = '/kaggle/working/nornikel-2024/'
rag_input_dir = '/kaggle/input/nornikel-2024-rag'
rag_target_dir = '/kaggle/working/nornikel-2024-rag'

if not os.path.exists(working_folder):
    os.makedirs(working_folder)

Вся работа велась в Kaggle, а при подгрузке данных он автоматически распаковывает все архивы (`.zip`, `.tar.gz`, `.gz`), для загрузки конфигурационных файлов byaldi требует наличия именно заархифированных конфиг-файлов (то есть не `.json`, а `.json.gz`), поэтому перекинем все эмбеддинги документов в рабочий каталог и назад зашифруем конфиги.

In [3]:
def process_files(source, target):
    os.makedirs(target, exist_ok=True)

    for root, dirs, files in os.walk(source):
        rel_path = os.path.relpath(root, source)
        target_root = os.path.join(target, rel_path)

        os.makedirs(target_root, exist_ok=True)

        for filename in files:
            source_path = os.path.join(root, filename)
            target_path = os.path.join(target_root, filename)

            if filename.endswith('.json'):
                target_path = target_path.replace('.json', '.json.gz')
                with open(source_path, 'rb') as f_in:
                    with gzip.open(target_path, 'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)
            else:
                shutil.copy2(source_path, target_path)

process_files(rag_input_dir, rag_target_dir)

Сформируем словарь со всеми нашими документами, чтобы в дальнейшем было проще с ними работать.

In [4]:
file_path = '/kaggle/working/nornikel-2024-rag/nornikel_index/doc_ids_to_file_names.json.gz'
docs_names = dict()

with gzip.open(file_path, 'rt', encoding='utf-8') as f:
    docs_names = json.load(f)
    docs_names = {int(key): value for key, value in docs_names.items()}

In [5]:
def convert_doc_to_pdf(doc_path, pdf_path):
    document = Document()
    document.LoadFromFile(doc_path)
    document.SaveToFile(pdf_path, FileFormat.PDF)
    document.Close()

Переведём все наши `.pdf`-файлы в рабочий каталог, а файлы `.docx` переведём в требуемый для ColPali формат `.pdf`.

In [6]:
all_documents = os.listdir(input_folder)

for file in all_documents:
    file_path = os.path.join(input_folder, file)
    if file_path.endswith('.docx') or file_path.endswith('.doc'):
        file_path = convert_doc_to_pdf(file_path, working_folder + file[:-4] + 'pdf')
        print(f"Копирован и переведён в формат .pdf файл: {file[:-4] + 'pdf'}")
    else:
        shutil.copy(file_path, working_folder)
        print(f"Копирован файл: {file}")

Копирован файл: Alrosa_Обзор_рынка_инвестиционных_бриллиантов_октябрь_2024.pdf
Копирован файл: Росконгресс_Рынок_промышленных_роботов_в_мире_и_России_2024_16_стр.pdf
Копирован файл: nn_climate_change_report_rus.pdf
Копирован файл: Норникель про корп культуру.pdf
Копирован файл: ММК 2024.pdf
Копирован файл: Доклад, уголь часть 1.pdf
Копирован файл: Godovoi_-otchet-PAO-GMK-Norilskii_-nikel-za-2023-god.pdf
Копирован файл: sr_ru_annual_report_pages_nornik_2022.pdf
Копирован файл: Норникель_Внутрення_цена_на_углерод.pdf
Копирован файл: 2_5282802846297776741.pdf
Копирован и переведён в формат .pdf файл: СП_496_1325800_2020_Основания_и_фундаменты_зданий_и_сооружений.pdf
Копирован файл: digital_production_5.pdf
Копирован файл: NN_AR_2021_Book_RUS_26.09.22.pdf
Копирован файл: McKinsey_Next Big Arenas_2024 (213 pgs).pdf
Копирован файл: KPMG_Global Metals and Mining_2024 (48 pgs).pdf
Копирован файл: NN_CSO2021_RUS_03.03.2023.pdf
Копирован файл: 2022_Annual_Report_of_PJSC_MMC_Norilsk_Nickel_rus.

Загрузим нашу готовую базу по индексу, который мы задали в первой части ноутбука, переведём её в работу на `cuda:0`.

In [7]:
RAG = RAGMultiModalModel.from_index(
    index_path='nornikel_index/',
    index_root='/kaggle/working/nornikel-2024-rag',
    device='cuda:0'
)

Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


adapter_config.json:   0%|          | 0.00/750 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/243k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/byaldi/colpali.py:162: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.indexed_embeddings.extend(torch.load(file))


По текстовому запросу определим k релевантных документов.

In [56]:
text_query = "Расскажи про корпоративную культуру в компании Норникель"
results = RAG.search(text_query, k=3)

In [57]:
for result in results:
    doc_id = result['doc_id']
    page_num = result['page_num']
    score = result['score']

    print(f"Документ ID: {doc_id}, Страница: {page_num}, Релевантность: {score}")

Документ ID: 26, Страница: 4, Релевантность: 18.125
Документ ID: 26, Страница: 6, Релевантность: 17.75
Документ ID: 26, Страница: 8, Релевантность: 17.75


# Qwen-2-VL

Так как ColPali не считывает напрямую текст с документов и изображений, нам потребуется языковая LLM или VLM. Для ответа по релевантным страницам документов и текстовому запросу используем VLM-модель `Qwen-2-VL-2B`, а именно дообученную на русских текстах версию от Vikhrmodels: `Vikhr-2-VL-2b-Instruct-experimental`.

In [10]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

model_id = "Vikhrmodels/Vikhr-2-VL-2b-Instruct-experimental"

model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_id, torch_dtype="auto", device_map="cuda:1"
)

processor = AutoProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.42G [00:00<?, ?B/s]

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Ниже мы берём релевантные страницы разных документов и переводим их в изображения. С целью оптимизации потребрения оперативной памяти снизим размер изображения примерно до 800 пикселей в длине.

In [19]:
def resize_text(img, max_size=800):
    width, height = img.size
    if max(width, height) > max_size:
        if width > height:
            new_width = max_size
            new_height = int(height * (max_size / width))
        else:
            new_height = max_size
            new_width = int(width * (max_size / height))
            
        img = img.resize((new_width, new_height), Image.LANCZOS)
        img = img.filter(ImageFilter.SHARPEN)

    return img

In [58]:
def get_relevant_images_ready(docs_names, results):
    images = [docs_names[results[i]['doc_id']] for i in range(len(results))]
    image_path = '/kaggle/working/nornikel-2024/relevant_images/'
    
    os.makedirs(image_path, exist_ok=True)
    pages_to_convert = dict()
    
    for doc, page in zip(images, [doc['page_num'] for doc in results]):
        pages_to_convert.setdefault(doc, []).append(page)
    
    ready_images_for_context = []
    
    for i, (pdf, pages) in enumerate(pages_to_convert.items()):
        for j, page in enumerate(pages):
            output_img_name = f'{image_path}/image_{i + 1}_{j + 1}.jpg'
            
            image = convert_from_path(pdf, first_page=page, last_page=page)
            image = resize_text(*image)
            image.save(output_img_name, 'JPEG')

            ready_images_for_context.append(output_img_name)
    

    return ready_images_for_context

relevant_images = get_relevant_images_ready(docs_names, results)

Формируем контент для контектного окна, куда мы передаём наш текстовый запрос и пути по готовых изображений.

In [59]:
content = [
    *[{"type": "image", "image": image} for image in relevant_images],
    {"type": "text", "text": text_query}
]

In [60]:
messages = [
    {"role": "system", "content": [{"type": "text", "text": "Ты бизнес-ассистент. Отвечай на вопрос точно."}]},
    {
        "role": "user",
        "content": content,
    }
]

display(messages)

[{'role': 'system',
  'content': [{'type': 'text',
    'text': 'Ты бизнес-ассистент. Отвечай на вопрос точно.'}]},
 {'role': 'user',
  'content': [{'type': 'image',
    'image': '/kaggle/working/nornikel-2024/relevant_images//image_1_1.jpg'},
   {'type': 'image',
    'image': '/kaggle/working/nornikel-2024/relevant_images//image_1_2.jpg'},
   {'type': 'image',
    'image': '/kaggle/working/nornikel-2024/relevant_images//image_1_3.jpg'},
   {'type': 'text',
    'text': 'Расскажи про корпоративную культуру в компании Норникель'}]}]

In [61]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda:1")

После передачи текстового запроса в модель получаем готовый сгенерированный моделью ответ.

In [62]:
generated_ids = model.generate(
    **inputs,
    max_length=2048,
    temperature=0.3,
    top_k=100,
    top_p=0.95
)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text[0])

Мы говорим о предпочтениях и изменениях в бизнесе, которые вносят вклад в культуру компании, а также о роли руководства в процессе формирования и оптимизации культуры компании. Критерии оценки включают: уровень готовности, уровень сотрудничества, уровень развития, уровень обучения, уровень коммуникации и уровень успеха.
